# Matrix Equivalents from the Hypercomplex Algebra

Two numbers from a hypercomplex algebra $z, w \in \mathbb{H}^N$ can be multiplied such that the result $m = zw, m \in \mathbb{H}^N$ is an element of the same algebra. The space is closed under addition as well, and so hypercomplex algebras are also vector spaces.

The map between an elementwise product of two hypercomplex vector  $\hat{z}, \hat{w} \in \mathbb{H}^{M \times N}$ can be viewed as as a linear transformation on the elements of the right factor of the product $\hat{m} = \hat{z} \hat{w} = A \hat{w}$.

Here the matrix $A$ depends on the entries of the geometric vector $\hat{z}$. In this notebook, we explore a systematic way to generate these transformation matrices.

$\hat{z}, \hat{w} \in \mathbb{H}^{M \times N}$

$\hat{m} \in \mathbb{H}^{M \times N}, \hat{z} \hat{w} = \hat{m} $

frame as a linear transformation with matrix $A_{[\hat{z}]}$ conitioned on $\hat{z}$

$ A_{[\hat{z}]} \hat{w} = \hat{m} $ 

estimate $A_{[\hat{z}]}$ given $\hat{w}, \hat{m}$ with a static $\hat{z}$ by randomly sampling $\hat{w}$ and computing $\hat{m}$

$ A_{[\hat{z}]}[t] \in \mathbb{R}^{N \times N}, A_{[\hat{z}]}[t + 1] = (\hat{y} \hat{x}^{T} + \gamma A_{[\hat{z}]}[t])(\hat{x} \hat{x}^{T} + \gamma I)^{-1} $

reduce equation in terms of $A_{[\hat{z}]}[0]$

$ A_{[\hat{z}]}[t] = \gamma^{t} A_{[\hat{z}]}[0] (\hat{x} \hat{x}^{T} + \gamma I)^{-t} + \sum_{i = 1}^{t} \gamma^{i - 1} \hat{y} \hat{x}^{T} (\hat{x} \hat{x}^{T} + \gamma I)^{-i} $

assume $ |\gamma| < 1 $ and take limit $ t \rightarrow \infty $

$ A_{[\hat{z}]}[t \rightarrow \infty] = \sum_{i = 1}^{\infty} \gamma^{i - 1} \hat{y} \hat{x}^{T} (\hat{x} \hat{x}^{T} + \gamma I)^{-i} $

does not depend on the initial choice of $A_{[\hat{z}]}[0]$

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
def hypercomplex_conjugate(a):
    c = np.ones(a.shape)
    c[..., 1:] *= -1
    return c * a
def hypercomplex_multiply(a, b):
    if a.shape[-1] == 1:
        return a * b
    else:
        def cayley_dickson(p, q, r, s):
            return np.concatenate([
                (hypercomplex_multiply(
                    p,
                    r) -
                hypercomplex_multiply(
                    hypercomplex_conjugate(s),
                    q)),
                (hypercomplex_multiply(
                    s,
                    p) +
                hypercomplex_multiply(
                    q,
                    hypercomplex_conjugate(r))),
            ], axis=(len(a.shape) - 1))
        return cayley_dickson(
            a[..., :(a.shape[-1] // 2)],
            a[..., (a.shape[-1] // 2):],
            b[..., :(a.shape[-1] // 2)],
            b[..., (a.shape[-1] // 2):])

In [4]:
def hypercomplex_conjugate_gradient(a, da):
    return hypercomplex_conjugate(da)   
def hypercomplex_multiply_gradient(a, b, da, db):
    return (hypercomplex_multiply(da, b),
        hypercomplex_multiply(a, db))
def hypercomplex_basis_gradient(a):
    basis = np.zeros((a.shape[-1], a.shape[-1]))
    np.fill_diagonal(basis, 1)
    basis = basis.reshape((1, a.shape[-1], a.shape[-1]))
    return basis

In [5]:
class HCX(object):
    def random(*kdims, hcx_size=1, mean=0, std=1):
        shape = [
            kdims[i] if i < len(kdims) 
            else 2**hcx_size
            for i in range(len(kdims) + 1)]
        return np.random.normal(mean, std, shape)
    def basis(x, dx=0, dir=1):
        if dir > 0:
            return x
        else:
            return hypercomplex_basis_gradient(x)
    def conj(x, dx=0, dir=1):
        if dir > 0:
            return hypercomplex_conjugate(x)
        else:
            return hypercomplex_conjugate_gradient(x, dx)
    def add(x, y, dx=0, dy=0, dir=1):
        if dir > 0:
            return x + y
        else:
            return dx, dy
    def sub(x, y, dx=0, dy=0, dir=1):
        if dir > 0:
            return x - y
        else:
            return dx, -dy
    def mul(x, y, dx=0, dy=0, dir=1):
        if dir > 0:
            return hypercomplex_multiply(x, y)
        else:
            return hypercomplex_multiply_gradient(x, y, dx, dy)
    def norm(x, dx=0, dir=1):
        if dir > 0:
            return np.sum(
                hypercomplex_multiply(HCX.conj(x), x),
                axis=(len(x.shape) - 1))**0.5
        else:
            c = 0.5 / np.sum(
                hypercomplex_multiply(HCX.conj(x), x),
                axis=(x.shape[-1] - 1))**0.5
            g = hypercomplex_multiply_gradient(
                HCX.conj(x), 
                x, 
                HCX.conj(x, dx, dir=-1), 
                dx)
            r = HCX.conj(g[0]) + g[1]
            return c * r
    def inv(x, dx=0, dir=1):
        if dir > 0:
            return HCX.conj(x) / np.reshape(HCX.norm(x)**2, (-1, 1))
        else:
            return (HCX.conj(x, dx, dir=-1) 
                / HCX.norm(x)**2 
                - 2 * HCX.conj(x) 
                / HCX.norm(x)**3
                * HCX.norm(x, dx, dir=-1))

In [6]:
M = 2     # The hypercomplex size
N = 1   # The vector elements
T = 1e-20 # A convergenece threshold
L = 0.05  # A hyperparameter to tune
V = 100   # The number of validation steps
D = 100   # The number of iterations before validating


def validate(_a, _A):
    _loss = 0.0
    for i in range(V):
        _b = HCX.random(N, 1, hcx_size=M).transpose(0, 2, 1)
        _c = HCX.mul(
            _a.transpose(0, 2, 1), 
            _b.transpose(0, 2, 1)).transpose(0, 2, 1)
        _loss += np.sum(_c - np.matmul(_A, _b))**2
    return _loss / V


a = HCX.random(N, 1, hcx_size=M).transpose(0, 2, 1)
A = HCX.random(N, 2**M, hcx_size=M)
I = np.reshape(np.eye(2**M), (1, 2**M, 2**M))


loss = 1.0
iterations = 0
while loss > T:
    iterations += 1
    b = HCX.random(N, 1, hcx_size=M).transpose(0, 2, 1)
    c = HCX.mul(
        a.transpose(0, 2, 1), 
        b.transpose(0, 2, 1)).transpose(0, 2, 1)
    A = np.matmul(
        (c * b.transpose(0, 2, 1)) + L * A,
        np.linalg.inv(
            (b * b.transpose(0, 2, 1)) + L * I))
    if iterations % D == 0:
        loss = validate(a, A)
        print("Validation Loss:", loss)
    

print("Iterations until Convergence:", iterations)
print(A)

Validation Loss: 1.38828939682e-14
Validation Loss: 3.58410744852e-27
Iterations until Convergence: 200
[[[ 2.05103854 -2.41028615 -0.54923662  1.17618732]
  [ 2.41028615  2.05103854  1.17618732  0.54923662]
  [ 0.54923662 -1.17618732  2.05103854 -2.41028615]
  [-1.17618732 -0.54923662  2.41028615  2.05103854]]]


In [7]:
print(np.linalg.det(A)) # This matrix is full rank
eigen_values, eigen_vectors = np.linalg.eig(A)

[ 136.92079293]


In [8]:
print(eigen_values) # The eigenvalues have a particular structure

[[ 2.05103854+2.73761882j  2.05103854-2.73761882j  2.05103854+2.73761882j
   2.05103854-2.73761882j]]


In [9]:
print(eigen_vectors) # The eigenvectors have a particular structure

[[[-0.03779313-0.64304569j -0.03779313+0.64304569j  0.20670594-0.48050617j
    0.20670594+0.48050617j]
  [-0.70039117+0.j         -0.70039117-0.j         -0.55639543+0.j
   -0.55639543-0.j        ]
  [ 0.04002650-0.22277876j  0.04002650+0.22277876j -0.33940515-0.43924391j
   -0.33940515+0.43924391j]
  [ 0.08013593-0.19199444j  0.08013593+0.19199444j -0.18028012+0.27600471j
   -0.18028012-0.27600471j]]]
